# Modelo predictivo

### Importar paquetes

### Limpieza de los datos

In [1]:
import pandas as pd

Empezaremos por todas las transformaciones necesarias tras el análisis previo. El objetivo es conseguir un conjunto de entrenamiento y test para nuestros modelos.

A pesar de que el reto específica que se nos proporcionarían diferentes conjuntos de test, tan solo contamos con uno. Eso nos hizo sospechar, tras un examen manual, nos dimos cuenta que los lotes de test están mezclados con el resto. Así que lo primero será separar dichos lotes y a partir de ahi construir la tabla con los campos relevantes. 

Este proceso será iterativo, por lo que complica la entrega, ya que solo podemos entregar un archivo. Por tanto se irá documentando, las diferentes transformaciones que se harán en distintos puntos del proceso hasta conseguir un modelo con buen rendimiento.

Como primer acercamiento será obtener los valores agregados de las distintas fases para poder predecir la variable objetivo.
Para ello necesitaremos crear un `pipeline` que garantize el orden de los pasos, dado que la entrada que se nos proporciona para el test es insuficiente para realizar una predicción

In [2]:
df_cultivo_test = pd.read_excel('raw_data/Fases producción_test v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo_test.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,24054,1,NaN,14616,2024-04-16 08:12:00,2024-04-18 07:28:00,81.6,15.44,85.60,184800000,14246.0,27.84,23.96,NaN,NaN
1,24055,1,NaN,14614,2024-04-13 08:18:00,2024-04-15 08:30:00,NaN,14.32,73.68,175200000,12912.0,30.96,23.16,NaN,NaN
2,24056,1,NaN,14615,2024-04-13 08:18:00,2024-04-15 08:15:00,NaN,14.56,82.40,168000000,14246.0,29.52,28.88,NaN,NaN
3,24057,1,NaN,13170,2024-04-16 08:12:00,2024-04-18 07:41:00,82.4,17.76,78.96,180800000,12912.0,31.04,25.32,NaN,NaN
4,24058,2,24055.0,14614,2024-04-15 12:28:00,2024-04-17 08:14:00,87.2,18.00,82.40,144800000,12912.0,26.08,20.36,NaN,NaN


In [3]:
lotes_test = df_cultivo_test['LOTE'].unique()
len(lotes_test)

56

In [4]:
df_cultivo = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Cultivo final', engine='openpyxl')
df_cultivo.head(5)

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2
0,23019,1,NaN,14615,2023-03-21 07:30:00,2023-03-23 06:30:00,82.4,17.28,91.2,184000000,17825,NaN,NaN,1747.920,6.00
1,23020,1,NaN,14616,2023-03-21 07:30:00,2023-03-23 06:30:00,80.4,18.80,91.2,181600000,14246,NaN,NaN,1676.160,6.56
2,23021,1,NaN,13170,2023-03-22 07:30:00,2023-03-24 06:30:00,66.4,16.16,86.4,248000000,17825,NaN,NaN,1928.496,8.08
3,23022,1,NaN,14614,2023-03-22 07:30:00,2023-03-24 06:30:00,85.6,18.48,83.2,229600000,12912,NaN,NaN,1782.800,5.92
4,23023,1,NaN,14615,2023-03-28 07:27:00,2023-03-30 10:00:00,77.6,17.12,74.4,132800000,17825,26.56,20.88,1861.840,2.96


In [5]:
df_cultivo[df_cultivo['LOTE'].isin(lotes_test)]

,LOTE,Orden en el encadenado,LOTE parental,ID Bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de inóculo utilizado,Turbidez inicio cultivo,Turbidez fin cultivo,Viabilidad final cultivo,ID Centrífuga,Centrifugación 1 turbidez,Centrifugación 2 turbidez,Producto 1,Producto 2


In [6]:
df_preinoculo = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Preinóculo', header=[0, 1], na_values=['NA', 'N.A'], engine='openpyxl')
df_preinoculo.head(5)

Unnamed: 0_level_0  Unnamed: 1_level_0  Unnamed: 2_level_0      pH          \
                LOTE   Fecha/hora inicio      Fecha/hora fin línea 1 línea 2   
0              23023 2023-03-26 05:00:00 2023-03-27 07:21:00   5.496   5.504   
1              23024 2023-03-26 05:00:00 2023-03-27 07:21:00   5.496   5.504   
2              23025 2023-03-17 06:00:00 2023-03-28 07:42:00   5.480   5.520   
3              23026 2023-03-17 06:00:00 2023-03-28 07:42:00   5.480   5.520   
4              23027 2023-04-02 05:00:00 2023-04-03 13:30:00   5.384   5.440   

          Turbidez                 Línea utilizada                  
  línea 3  línea 1 línea 2 línea 3         línea 1 línea 2 línea 3  
0    5.52    28.32   27.92   32.00               1       1       0  
1    5.52    28.32   27.92   32.00               1       1       0  
2    5.52    26.56   27.52   26.40               1       1       0  
3    5.52    26.56   27.52   26.40               1       1       0  
4    5.40    33.84   30.56   32.48               1       0       1

In [7]:
df_preinoculo[df_preinoculo[('Unnamed: 0_level_0','LOTE')].isin(lotes_test)]

Unnamed: 0_level_0  Unnamed: 1_level_0  Unnamed: 2_level_0      pH  \
                  LOTE   Fecha/hora inicio      Fecha/hora fin línea 1   
130              24055 2024-04-11 00:30:00 2024-04-12 07:14:00   5.368   
131              24056 2024-04-11 00:30:00 2024-04-12 07:14:00   5.368   
132              24054 2024-04-14 00:30:00 2024-04-15 07:17:00   5.504   
133              24057 2024-04-14 00:30:00 2024-04-15 07:17:00   5.504   
134              24060 2024-04-18 00:30:00 2024-04-19 07:46:00   5.384   
135              24061 2024-04-18 00:30:00 2024-04-19 07:46:00   5.384   
136              24063 2024-04-21 00:30:00 2024-04-22 07:19:00   5.360   
137              24064 2024-04-21 00:30:00 2024-04-22 07:19:00   5.360   
138              24067 2024-04-25 00:30:00 2024-04-26 07:17:00   5.568   
139              24070 2024-04-25 00:30:00 2024-04-26 07:17:00   5.568   
140              24071 2024-04-28 00:30:00 2024-04-29 07:45:00   6.280   
141              24072 2024-05-12 00:30:00 2024-05-13 08:16:00   5.424   
142              24073 2024-05-12 00:30:00 2024-05-13 07:54:00   5.440   
143              24074 2024-05-16 00:30:00 2024-05-17 07:15:00   5.408   
144              24075 2024-05-12 00:30:00 2024-05-13 07:54:00   5.424   
145              24076 2024-05-16 00:30:00 2024-05-17 07:15:00   5.408   
146              24077 2024-05-19 00:30:00 2024-05-20 07:14:00   5.400   
147              24078 2024-05-19 00:30:00 2024-05-20 07:14:00   5.400   
148              24081 2024-05-26 00:30:00 2024-05-27 08:05:00   5.440   
149              24083 2024-05-23 00:30:00 2024-05-24 07:26:00   5.384   
150              24084 2024-05-23 00:30:00 2024-05-24 07:26:00   5.384   
151              24085 2024-05-26 00:30:00 2024-05-27 08:05:00   5.440   
152              24088 2024-05-30 00:30:00 2024-05-31 07:46:00   5.472   
153              24089 2024-05-30 00:30:00 2024-05-31 07:46:00   5.472   
154              24091 2024-06-02 00:30:00 2024-06-03 07:03:00   5.392   
155              24092 2024-06-02 00:30:00 2024-06-03 07:03:00   5.392   
156              24095 2024-06-09 00:30:00 2024-06-10 07:09:00   5.456   
157              24097 2024-06-09 00:30:00 2024-06-10 07:09:00   5.456   
158              24098 2024-06-13 00:30:00 2024-06-14 08:00:00   5.672   
159              24101 2024-06-27 00:30:00 2024-06-28 07:16:00   5.424   
160              24103 2024-06-27 00:30:00 2024-06-28 07:16:00   5.424   
161              24104 2024-06-30 00:30:00 2024-07-01 07:01:00   5.408   
162              24105 2024-06-30 00:30:00 2024-07-01 07:01:00   5.408   
163              24108 2024-07-04 00:30:00 2024-07-05 07:04:00   5.528   

                    Turbidez                 Línea utilizada                  
    línea 2 línea 3  línea 1 línea 2 línea 3         línea 1 línea 2 línea 3  
130   5.416   5.464    26.16   25.52   25.04               1       1       0  
131   5.416   5.464    26.16   25.52   25.04               1       1       0  
132   5.376   5.376    27.12   25.68   29.52               1       1       0  
133   5.376   5.376    27.12   25.68   29.52               1       1       0  
134   5.344   5.448    25.44   26.00   24.72               1       1       0  
135   5.344   5.448    25.44   26.00   24.72               1       1       0  
136   5.376     NaN    25.84   27.84     NaN               1       1       0  
137   5.376     NaN    25.84   27.84     NaN               1       1       0  
138   5.560   5.520    20.96   22.48   22.40               0       1       1  
139   5.560   5.520    20.96   22.48   22.40               0       1       1  
140   5.488     NaN    27.36   24.88     NaN               0       1       0  
141   5.424     NaN    25.36   30.96     NaN               0       1       0  
142   5.384   5.456    25.12   26.96   27.84               0       1       1  
143   5.408   5.384    27.36   28.08   24.32               1       1       0  
144   5.424     NaN    25.12   26.96   27.84               0       1       

___

In [14]:
def get_OF_dataset():
    df = pd.read_excel('raw_data/OF 123456 v02.xlsx', engine='openpyxl')
    df['LOTE'] = df['Lote'].apply(lambda x: int(x.replace('/', '').replace('P', '')))
    return df[['Orden','LOTE']]

In [72]:
def get_preinoculo_dataset(lotes):
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Preinóculo', header=[0, 1], na_values=['NA', 'N.A'], engine='openpyxl')
    new_cols = [l2 if 'Unnamed' in l1 else f"{l1}-{l2}" for l1, l2 in df.columns.to_list()]
    df.columns = new_cols
    # Filtro de lotes
    df = df[df['LOTE'].isin(lotes)]
    # Imputación
    for c in df.select_dtypes(include='float').columns:
        df[c] = pd.to_numeric(df[c], errors='coerce')
        df[c] = df[c].fillna(df[c].mean())
    # Media de valores
    df['ph'] = (
        (
            df[['pH-línea 1', 'pH-línea 2', 'pH-línea 3']].values *
            df[['Línea utilizada-línea 1', 'Línea utilizada-línea 2', 'Línea utilizada-línea 3']].values
        ).sum(axis=1) /
        df[['Línea utilizada-línea 1', 'Línea utilizada-línea 2', 'Línea utilizada-línea 3']].sum(axis=1)
    ).round(3)
    df['turbidez'] = (
        (
            df[['Turbidez-línea 1', 'Turbidez-línea 2', 'Turbidez-línea 3']].values *
            df[['Línea utilizada-línea 1', 'Línea utilizada-línea 2', 'Línea utilizada-línea 3']].values
        ).sum(axis=1) /
        df[['Línea utilizada-línea 1', 'Línea utilizada-línea 2', 'Línea utilizada-línea 3']].sum(axis=1)
    ).round(2)
    # Calculo duración
    df['duracion_horas'] = (df['Fecha/hora fin'] - df['Fecha/hora inicio']).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, ['Fecha/hora inicio', 'Fecha/hora fin']] = df.loc[df['duracion_horas'] < 0, ['Fecha/hora fin', 'Fecha/hora inicio']].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)
    return df[['LOTE','ph','turbidez','duracion_horas']]


In [147]:
def get_inoculo_dataset(lotes, preinoculo):
    df = pd.read_excel('raw_data/Fases producción v02.xlsx', sheet_name='Inóculo', engine='openpyxl')
    df['Viabilidad final cultivo'] = pd.to_numeric(df['Viabilidad final cultivo'], errors='coerce')
    # Filtrado
    df = df[df['LOTE'].isin(lotes)]
    df.dropna(subset=['Fecha/hora inicio','Fecha/hora fin'], inplace=True)
    # Imputación
    df.loc[df['Turbidez inicio cultivo'].isna(), ['Turbidez inicio cultivo']] = df[df['Turbidez inicio cultivo'].isna()].join(preinoculo, on='LOTE', how='left', rsuffix='preinoculo')['turbidez']
    df['diferencia_turbidez'] = df['Turbidez final culttivo'] - df['Turbidez inicio cultivo']
    # > Si con la anterior imputación volvemos a tener un NaN en la resta, imputamos por la media
    df['diferencia_turbidez'] = df['diferencia_turbidez'].fillna(df['diferencia_turbidez'].mean())
    
    df['duracion_horas'] = (df['Fecha/hora fin'] - df['Fecha/hora inicio']).dt.total_seconds() / 3600
    df.loc[df['duracion_horas'] < 0, ['Fecha/hora inicio', 'Fecha/hora fin']] = df.loc[df['duracion_horas'] < 0, ['Fecha/hora fin', 'Fecha/hora inicio']].values
    df['duracion_horas'] = df['duracion_horas'].abs().round(2)
    df = df.rename(columns={'Fecha/hora inicio': 'ts_inicio', 'Fecha/hora fin': 'ts_fin', 'ID bioreactor': 'id_bioreactor'})
    return df[['LOTE', 'id_bioreactor', 'ts_inicio', 'ts_fin', 'diferencia_turbidez', 'duracion_horas']]

In [148]:
df_info_general = get_OF_dataset()
lotes = df_info_general['LOTE'].unique()
preinoculo = get_preinoculo_dataset(lotes)
inoculo = get_inoculo_dataset(lotes, preinoculo)

In [150]:
get_inoculo_dataset(lotes, preinoculo)

,LOTE,id_bioreactor,ts_inicio,ts_fin,diferencia_turbidez,duracion_horas
4,23023,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
5,23024,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,13.44,24.12
6,23025,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
7,23026,13172,2023-03-28 07:42:00,2023-03-29 06:43:00,10.64,23.02
8,23027,13172,2023-04-03 13:30:00,2023-04-04 11:35:00,4.64,22.08
...,...,...,...,...,...,...
163,24101,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
164,24103,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,11.92,23.83
165,24104,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,11.92,25.00
166,24105,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,11.92,25.00
